In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.226058,"""a""",0.708591,0.462704,0.698987,0.356959,0.350674,-0.803339,0.230494
0.841471,1,"""a""",0,0.523528,"""a""",0.123305,0.724855,0.171432,0.868455,0.910366,-0.021179,0.325117
0.909297,2,"""a""",1,0.568643,"""a""",0.996366,0.722279,0.028079,0.624149,0.223698,0.324061,0.451812
0.14112,3,"""a""",1,0.187578,"""a""",0.518841,0.262821,0.643063,0.013103,0.408594,-0.80789,0.121423
-0.756802,4,"""a""",0,0.970416,"""a""",0.698661,0.704845,0.377904,0.71565,0.83899,-0.250571,0.370264


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.356959,-0.350674
-0.841471,-0.868455,-0.910366
-0.909297,-0.624149,-0.223698
-0.14112,-0.013103,-0.408594
0.756802,-0.358691,-0.488315


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.503983, -0.350111]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.503983, -0.350111]","[0.086441, 0.406699]"


In [9]:
# If you want the underlying calculation to be done in f32 instead of f64, you may use the following.
# In some cases, f32 can run faster, especially when input data is in f32.
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f32],list[f32]
"[-0.503983, -0.350111]","[0.086441, 0.406699]"


In [10]:
pds.config.LIN_REG_EXPR_F64 = True

In [11]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f64,f64,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.220117,0.001687,130.489333,0.0,0.216811,0.223424,0.994412,0.99441
"""exp(x2)""",0.175348,0.000679,258.255791,0.0,0.174017,0.176679,0.994412,0.99441
"""sin(x3)""",-1.745157,0.00134,-1301.974452,0.0,-1.747785,-1.74253,0.994412,0.99441
"""__bias__""",-0.109283,0.001507,-72.499729,0.0,-0.112238,-0.106328,0.994412,0.99441


In [12]:
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f32,f32,f32,f32,f32,f32,f32,f32
"""ln(x1+1)""",0.220121,0.001687,130.491577,0.0,0.216815,0.223428,0.994412,0.99441
"""exp(x2)""",0.175347,0.000679,258.253998,0.0,0.174016,0.176678,0.994412,0.99441
"""sin(x3)""",-1.745157,0.00134,-1301.974243,0.0,-1.747784,-1.742529,0.994412,0.99441
"""__bias__""",-0.109282,0.001507,-72.499084,0.0,-0.112237,-0.106327,0.994412,0.99441


In [13]:
pds.config.LIN_REG_EXPR_F64 = True

In [14]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.512962, -0.340556]"
"""a""","[-0.512962, -0.340556]"
"""a""","[-0.512962, -0.340556]"
"""a""","[-0.512962, -0.340556]"
"""a""","[-0.512962, -0.340556]"
…,…
"""b""","[-0.49521, -0.359508]"
"""b""","[-0.49521, -0.359508]"
"""b""","[-0.49521, -0.359508]"


In [15]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.708591,0.462704,-0.803339,-0.519115,-0.284224
0.123305,0.724855,-0.021179,-0.315923,0.294744
0.996366,0.722279,0.324061,-0.755029,1.079091
0.518841,0.262821,-0.80789,-0.353503,-0.454386
0.698661,0.704845,-0.250571,-0.598887,0.348316


In [16]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.512962, -0.340556]"
"""b""","[-0.49521, -0.359508]"


In [17]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[-0.319952, -0.189272]"
"""a""","[-0.336547, -0.174777]"


In [18]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.569388
"""b""",-0.51467


In [19]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.803339,0.708591,0.462704,null,null
-0.021179,0.123305,0.724855,null,null
0.324061,0.996366,0.722279,null,null
-0.80789,0.518841,0.262821,null,null
-0.250571,0.698661,0.704845,"[-0.569476, 0.253733]",-0.219029
…,…,…,…,…
0.155725,0.955219,0.908676,"[-1.11676, 0.508952]",-0.604278
-0.253023,0.602168,0.733212,"[-1.193864, 0.812388]",-0.123254
0.108824,0.737362,0.713642,"[0.973572, -1.126835]",-0.086281


In [20]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [21]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.226961, 28.83222, 28.429692]"


In [22]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.851301,"[0.911501, -0.411297]"
28.806003,"[0.411297, 0.911501]"


In [23]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.070858
0.165171
0.22491
-0.408106
0.055246


# ML Metrics

In [24]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.333839,1.006882,0.511637,0.509427,0.510529,0.509782,0.503323
"""b""",0.33568,1.004785,0.499603,0.498812,0.499208,0.505857,0.496367


# Str Extension

In [25]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [26]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

/tmp/ipykernel_13570/3113574353.py:3: DeprecationWarning: This function is deprecated. Please use `pl.col(..).str.extract_all(pattern)`, where pattern can be r'(?u\w\w'.
  pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()


sen
str
"""to"""
"""world"""
"""church"""
"""hello"""
"""going"""


In [27]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [28]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [29]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [30]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [31]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [32]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [33]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [34]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [35]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.235002
0.690898
-2.766704


In [36]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,1.070217,null
null,-0.571811,null
-0.235002,0.736468,-0.470783
0.690898,1.677268,1.808989
-2.766704,1.314463,0.62103


In [37]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""oQn8Y""",null
null,"""XZV3""",null
-0.235002,"""Qm""","""6ZpT"""
0.690898,"""DUVaG""","""T"""
-2.766704,"""2bg""","""ZO"""


In [38]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.235002,"""DCR5x"""
0.690898,"""PqEEc"""
-2.766704,"""UYb7X"""


In [39]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.123625,null,-0.123441
null,-1.691028,null,-1.691397
-0.235002,0.856147,0.524302,0.856287
0.690898,-1.471031,1.066612,-1.47093
-2.766704,-0.373147,0.049235,-0.37355


In [40]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.143557,0.374139,8
null,0.424823,2.333433,5
-0.235002,0.193159,0.64696,8
0.690898,0.083981,2.059376,2
-2.766704,0.553186,0.532175,8


In [41]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.508863,0.610926,0.262524,0.876988


In [42]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.672702,0.865685,0,8
1,0.705295,0.563739,0,7
2,0.58488,0.081444,1,3
0,0.67017,0.38607,3,1
1,0.4255,0.084388,2,5


In [43]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.44077,0.659389}","{32.159171,0.651894}","{1.629011,0.163967}"


In [44]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.678266,0.49765}   ┆ {27.885553,0.831255} ┆ {0.527716,0.71539}  │
│ 1         ┆ {0.494796,0.620777}  ┆ {31.501982,0.6824}   ┆ {2.438253,0.045223} │
│ 2         ┆ {-0.418066,0.675925} ┆ {28.162548,0.821357} ┆ {2.882355,0.021516} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [45]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
559,0.1118
561,0.1122
542,0.1084
555,0.111
548,0.1096
551,0.1102
552,0.1104
557,0.1114
575,0.115


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [46]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [47]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.876859,0.133115,0.423981,0.503255,7.166269,12
1,0.994173,0.428538,0.904636,0.443848,0.502891,10
2,0.581176,0.818299,0.717389,0.836512,4.573288,19
3,0.377086,0.714554,0.191054,0.558064,3.178318,18
4,0.710425,0.80805,0.756021,0.162609,3.843696,20


In [48]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.876859,0.133115,0.423981,0.503255,7.166269,228
1,0.994173,0.428538,0.904636,0.443848,0.502891,85
2,0.581176,0.818299,0.717389,0.836512,4.573288,892
3,0.377086,0.714554,0.191054,0.558064,3.178318,383
4,0.710425,0.80805,0.756021,0.162609,3.843696,13


In [49]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.876859,0.133115,0.423981,0.503255,7.166269,"[0, 1524, … 1946]"
1,0.994173,0.428538,0.904636,0.443848,0.502891,"[1, 39, … 125]"
2,0.581176,0.818299,0.717389,0.836512,4.573288,"[2, 380, … 1174]"
3,0.377086,0.714554,0.191054,0.558064,3.178318,"[3, 685, … 496]"
4,0.710425,0.80805,0.756021,0.162609,3.843696,"[4, 265, … 1106]"


In [50]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1524, … 1946] ┆ 3                  │
│ 1   ┆ [1, 39, … 1422]   ┆ 7                  │
│ 2   ┆ [2, 380, … 1517]  ┆ 11                 │
│ 3   ┆ [3, 685, … 944]   ┆ 9                  │
│ 4   ┆ [4, 265, … 1002]  ┆ 9                  │
└─────┴───────────────────┴────────────────────┘


In [51]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.876859,0.133115,0.423981,0.503255,7.166269,"[0, 1524, … 1946]","[0.0, 0.072463, … 0.083246]"
1,0.994173,0.428538,0.904636,0.443848,0.502891,"[1, 39, … 125]","[0.0, 0.060109, … 0.082179]"
2,0.581176,0.818299,0.717389,0.836512,4.573288,"[2, 380, … 1174]","[0.0, 0.032803, … 0.059227]"
3,0.377086,0.714554,0.191054,0.558064,3.178318,"[3, 685, … 496]","[0.0, 0.030836, … 0.05278]"
4,0.710425,0.80805,0.756021,0.162609,3.843696,"[4, 265, … 1106]","[0.0, 0.057415, … 0.061693]"


In [52]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
2,0.581176,0.818299,0.717389,0.836512,4.573288
3,0.377086,0.714554,0.191054,0.558064,3.178318
6,0.501085,0.446665,0.669058,0.295177,3.797496
10,0.4565,0.334125,0.541461,0.240152,6.451545
13,0.152949,0.629336,0.648148,0.099689,6.246647


In [53]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
6,0.501085,0.446665,0.669058,0.295177,3.797496
12,0.517317,0.574726,0.997668,0.336344,1.81484
30,0.483279,0.482538,0.249474,0.617154,7.513426
81,0.50054,0.568065,0.464834,0.428289,2.536523
168,0.436916,0.509921,0.079442,0.256517,7.261429


In [54]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
30,0.483279,0.482538,0.249474,0.617154,7.513426
168,0.436916,0.509921,0.079442,0.256517,7.261429
256,0.546873,0.525636,0.672437,0.719224,7.529955
298,0.509085,0.471096,0.925357,0.564083,5.556756
349,0.496174,0.560128,0.05123,0.811701,7.688234


In [55]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1274, … 1283]",4
1,"[1, 47]",2
2,"[2, 1427, 1652]",3
3,"[3, 685, … 676]",4
4,"[4, 890, … 24]",7


# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures.

In [56]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_13570/3354819425.py:3: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
1.0,0.621664,0,6,"""22""","""q"""
0.0,0.763334,0,1,"""X""","""aA"""
0.0,0.537238,2,0,"""ej""","""W0"""
1.0,0.330224,2,9,"""c""","""Yq"""
0.0,0.5436,0,7,"""JM""","""ZU"""


In [57]:
df_pd = df.to_pandas()

In [58]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

jaccard_col
f64
0.3


In [59]:
# Polars Series
print(pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest())
# Pyarrow Series
print(pds2.query_binary_metrics(df["actual"].to_arrow(), df["predicted"].to_arrow()).struct.unnest())
# NumPy
print(pds2.query_binary_metrics(df["actual"].to_numpy(), df["predicted"].to_numpy()).struct.unnest())

shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.499641  ┆ 0.502827 ┆ 0.501229 ┆ 0.50112           ┆ 0.502869 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.499641  ┆ 0.502827 ┆ 0.501229 ┆ 0.50112           ┆ 0.502869 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬───────

In [60]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.19793,0.2,0.217,0.001387
0.395133,0.2,0.201,0.000005
0.587547,0.2,0.202,0.00002
0.797317,0.2,0.193,0.000249
inf,0.2,0.187,0.000874


In [61]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

cid_ce
f64
12.568847


In [62]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

c3_stats
f64
0.114879


In [63]:
pds2.str_leven(df["s1"], df["s2"]).head() 

str_leven
u32
2
2
2
2
2
2
2
1
2
